In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data = pd.read_csv("data/training_data_utf8.csv")

/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head()

,id,price_per_m2,transaction_ym,plot_area,plot_share,property_type,cadastral,district,contract_type,land_use,land_use_share,protection_zone,building_ban,building_height,building_floors,building_year,longitude,latitude
0,7671,224.351484,2013-12,4279,1.0,Betriebsobjekt,Kagran,Donaustadt,Kaufvertrag,GB,1.00,No,No,NaN,1.0,1990.0,16.464163,48.259071
1,7672,357.995227,2013-12,838,1.0,Abbruchobjekt,Liesing,Liesing,Kaufvertrag,W,1.00,No,No,10.5,2.0,NaN,16.275103,48.133264
2,7673,636.474908,2013-12,817,1.0,"Ein-, Zweifamilienhaus",Atzgersdorf,Liesing,Kaufvertrag,GB,1.00,No,No,3.5,2.0,NaN,16.296926,48.145348
3,7674,906.095552,2013-12,607,1.0,"Ein-, Zweifamilienhaus",Ottakring,Ottakring,Kaufvertrag,W,1.00,No,No,7.5,2.0,NaN,16.290968,48.217580
4,7675,167.941498,2013-12,11897,1.0,unbebaut,Groß Jedlersdorf I,Floridsdorf,Kaufvertrag,W,0.19,No,No,NaN,NaN,NaN,16.418471,48.288753


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44366 entries, 0 to 44365
Data columns (total 18 columns):
id                 44366 non-null int64
price_per_m2       44366 non-null float64
transaction_ym     44366 non-null object
plot_area          44366 non-null int64
plot_share         44366 non-null float64
property_type      44366 non-null object
cadastral          44366 non-null object
district           44294 non-null object
contract_type      44366 non-null object
land_use           44260 non-null object
land_use_share     44320 non-null float64
protection_zone    44366 non-null object
building_ban       44366 non-null object
building_height    7800 non-null object
building_floors    18422 non-null float64
building_year      14739 non-null float64
longitude          32465 non-null float64
latitude           32465 non-null float64
dtypes: float64(7), int64(2), object(9)
memory usage: 6.1+ MB


In [5]:
#describe numerical values
data.describe()

,id,price_per_m2,plot_area,plot_share,land_use_share,building_floors,building_year,longitude,latitude
count,44366.000000,44366.000000,4.436600e+04,44366.000000,44320.000000,18422.000000,14739.000000,32465.000000,32465.000000
mean,29853.500000,789.062369,1.880226e+03,0.959631,0.956965,2.063402,1913.283737,16.363182,48.207192
std,12807.505358,2276.246856,1.064498e+04,0.163726,0.144435,1.902943,37.464120,0.074558,0.039031
min,7671.000000,0.161937,1.000000e+00,0.000260,0.000000,0.000000,1800.000000,15.605753,47.850748
25%,18762.250000,181.682084,4.120000e+02,1.000000,1.000000,0.000000,1894.000000,16.310912,48.179708
50%,29853.500000,354.343664,5.810000e+02,1.000000,1.000000,2.000000,1900.000000,16.352831,48.208886
75%,40944.750000,726.728429,1.035000e+03,1.000000,1.000000,4.000000,1940.000000,16.407386,48.230576
max,52036.000000,197628.458498,1.187781e+06,1.000000,1.000000,12.000000,2017.000000,16.572405,48.314311


In [6]:
#describe categorical values
data.describe(include="O")

,transaction_ym,property_type,cadastral,district,contract_type,land_use,protection_zone,building_ban,building_height
count,44366,44366,44366,44294,44366,44260,44366,44366,7800
unique,324,16,88,23,11,56,2,2,97
top,1988-12,unbebaut,Eßling,Donaustadt,Kaufvertrag,WI/GBI,No,No,6.5
freq,296,15345,2270,8476,38853,12099,43306,43857,2763


In [7]:
corrmat = data.corr()

In [8]:
corrmat["price_per_m2"].sort_values(ascending=False)

price_per_m2       1.000000
building_floors    0.385326
plot_share         0.002082
latitude          -0.011045
plot_area         -0.028618
longitude         -0.038409
land_use_share    -0.050230
building_year     -0.051976
id                -0.133134
Name: price_per_m2, dtype: float64

In [9]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

In [12]:
latitudeCenter = 48.210033
longitudeCenter = 16.363449

In [13]:
for index, row in data.iterrows():
    data.loc[index, 'distanceToCenter'] = haversine(row['longitude'], row['latitude'], longitudeCenter, latitudeCenter)

In [14]:
corrmat = data.corr()
corrmat["price_per_m2"].sort_values(ascending=False)

price_per_m2        1.000000
building_floors     0.385326
plot_share          0.002082
latitude           -0.011045
plot_area          -0.028618
longitude          -0.038409
land_use_share     -0.050230
building_year      -0.051976
id                 -0.133134
distanceToCenter   -0.268828
Name: price_per_m2, dtype: float64

In [16]:
data_fillna = data.fillna(data.mean())

In [19]:
corrmat_fillna = data_fillna.corr()
corrmat_fillna["price_per_m2"].sort_values(ascending=False)

price_per_m2        1.000000
building_floors     0.339758
plot_share          0.002082
latitude           -0.010008
plot_area          -0.028618
longitude          -0.034801
building_year      -0.040318
land_use_share     -0.050224
id                 -0.133134
distanceToCenter   -0.243580
Name: price_per_m2, dtype: float64

In [20]:
data_fillna.describe()

,id,price_per_m2,plot_area,plot_share,land_use_share,building_floors,building_year,longitude,latitude,distanceToCenter
count,44366.000000,44366.000000,4.436600e+04,44366.000000,44366.000000,44366.000000,44366.000000,44366.000000,44366.000000,44366.000000
mean,29853.500000,789.062369,1.880226e+03,0.959631,0.956965,2.063402,1913.283737,16.363182,48.207192,6.212574
std,12807.505358,2276.246856,1.064498e+04,0.163726,0.144360,1.226202,21.593079,0.063779,0.033388,2.811221
min,7671.000000,0.161937,1.000000e+00,0.000260,0.000000,0.000000,1800.000000,15.605753,47.850748,0.042117
25%,18762.250000,181.682084,4.120000e+02,1.000000,1.000000,2.000000,1913.283737,16.327764,48.192057,4.251330
50%,29853.500000,354.343664,5.810000e+02,1.000000,1.000000,2.063402,1913.283737,16.363182,48.207192,6.212574
75%,40944.750000,726.728429,1.035000e+03,1.000000,1.000000,2.063402,1913.283737,16.384375,48.220984,7.816682
max,52036.000000,197628.458498,1.187781e+06,1.000000,1.000000,12.000000,2017.000000,16.572405,48.314311,57.364157


In [22]:
data_dropna = data.dropna()
data_dropna.describe()

,id,price_per_m2,plot_area,plot_share,land_use_share,building_floors,building_year,longitude,latitude,distanceToCenter
count,2115.000000,2115.000000,2115.000000,2115.000000,2115.000000,2115.000000,2115.000000,2115.000000,2115.000000,2115.000000
mean,15068.195272,1330.217932,830.523404,0.975673,0.934525,2.666194,1930.382033,16.323705,48.183456,6.271457
std,4168.778817,1565.807231,1703.185982,0.119239,0.170043,1.410585,36.625488,0.051006,0.036735,2.998822
min,7704.000000,8.482564,6.000000,0.100000,0.030000,0.000000,1800.000000,16.214863,48.121595,0.623014
25%,11771.500000,450.073394,446.000000,1.000000,1.000000,2.000000,1900.000000,16.277842,48.153845,3.136859
50%,14762.000000,791.190865,565.000000,1.000000,1.000000,2.000000,1930.000000,16.324867,48.177829,6.977616
75%,18149.000000,1656.053643,790.500000,1.000000,1.000000,4.000000,1950.000000,16.354222,48.209042,8.759056
max,34835.000000,24691.358025,53021.000000,1.000000,1.000000,12.000000,2013.000000,16.535167,48.302999,13.665477


In [24]:
data_dropna.describe(include="O")

,transaction_ym,property_type,cadastral,district,contract_type,land_use,protection_zone,building_ban,building_height
count,2115,2115,2115,2115,2115,2115,2115,2115,2115
unique,145,12,79,23,5,12,2,2,41
top,2009-12,"Ein-, Zweifamilienhaus",Mauer,Liesing,Kaufvertrag,W,No,No,6.5
freq,36,994,224,601,2100,1686,2113,2114,608


In [23]:
corrmat_dropna = data_dropna.corr()
corrmat_dropna["price_per_m2"].sort_values(ascending=False)

price_per_m2        1.000000
building_floors     0.604109
latitude            0.319143
longitude           0.206471
plot_share         -0.049509
plot_area          -0.069220
land_use_share     -0.129005
id                 -0.139039
building_year      -0.215827
distanceToCenter   -0.515786
Name: price_per_m2, dtype: float64

In [26]:
building_height_num = data[data.building_height.apply(lambda x: x.isnumeric())]

AttributeError: 'float' object has no attribute 'isnumeric'